# Real Time Video Demo

## Importing Required Libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
import winsound
import mediapipe as mp

## Load the trained model And Defining Class Labels

In [2]:
model = tf.keras.models.load_model('models/Eye_Model2.h5')
class_names = ['Close-Eyes', 'Open-Eyes']

## Set sound frequency and duration

In [3]:
frequency = 2500  # Hz
duration = 2000   # 2 seconds

## Initialize MediaPipe face mesh

In [4]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

## Open webcam For A Real-Time Video Demo
- The code captures video from the webcam.
- Detects faces and eyes using OpenCV.
- Uses a deep learning model to classify eye state.
- Alerts the user if no face is detected for 3 seconds.
- Triggers a sleep alert if eyes are closed for too long.

1. Open Webcam

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

if not cap.isOpened():
    raise IOError("Cannot Open Webcam")

2. Initialize Variables

In [6]:
counter = 0
status = "Open Eyes"
face_detected_time = time.time()

3. Eye landmarks

In [7]:
LEFT_EYE_LANDMARKS = [33, 133, 153, 154, 155, 133]
RIGHT_EYE_LANDMARKS = [263, 362, 387, 386, 385, 362]
padding = 20

4. Define Functions

In [8]:
def get_eye_box(landmarks, landmark_indices, frame_shape):
    points = [(int(landmarks.landmark[i].x * frame_shape[1]),
               int(landmarks.landmark[i].y * frame_shape[0])) for i in landmark_indices]
    x_min = min(p[0] for p in points) - padding
    y_min = min(p[1] for p in points) - padding
    x_max = max(p[0] for p in points) + padding
    y_max = max(p[1] for p in points) + padding
    return (x_min, y_min, x_max, y_max)

def predict_eye_state(eye):
    if eye.size == 0: return 'Close-Eyes', 0.0
    eye_resized = cv2.resize(eye, (256, 256)) / 255.0
    eye_input = np.expand_dims(eye_resized, axis=0)
    prediction = model.predict(eye_input)
    return class_names[np.argmax(prediction[0])], np.max(prediction[0])


5. Main Loop

In [9]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks is None:
        if time.time() - face_detected_time > 3:
            cv2.putText(frame, 'Face Missing Alert!', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            winsound.Beep(frequency, duration)
    else:
        face_detected_time = time.time()
        for landmarks in results.multi_face_landmarks:
            left_x1, left_y1, left_x2, left_y2 = get_eye_box(landmarks, LEFT_EYE_LANDMARKS, frame.shape)
            right_x1, right_y1, right_x2, right_y2 = get_eye_box(landmarks, RIGHT_EYE_LANDMARKS, frame.shape)

            left_eye = frame[left_y1:left_y2, left_x1:left_x2]
            right_eye = frame[right_y1:right_y2, right_x1:right_x2]

            left_eye_status, _ = predict_eye_state(left_eye)
            right_eye_status, _ = predict_eye_state(right_eye)

            left_color = (0, 0, 255) if left_eye_status == 'Close-Eyes' else (0, 255, 0)
            right_color = (0, 0, 255) if right_eye_status == 'Close-Eyes' else (0, 255, 0)

            cv2.rectangle(frame, (left_x1, left_y1), (left_x2, left_y2), left_color, 2)
            cv2.rectangle(frame, (right_x1, right_y1), (right_x2, right_y2), right_color, 2)

            if left_eye_status == 'Close-Eyes' and right_eye_status == 'Close-Eyes':
                counter += 1
                status = "Closed Eyes"
            else:
                counter = 0
                status = "Open Eyes"

            cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if counter > 5:
                cv2.putText(frame, 'Sleep Alert !!', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                winsound.Beep(frequency, duration)
                counter = 0

    cv2.imshow("Real-Time Eye Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
